## Dense classifier for classification on MNIST dataset

This notebook trains a classifier for classifying the images of the classic MNIST handwritten digits dataset. This is implemented in an incremental manner, starting from a naive approach with bad performance. Every implementation addresses some issue in the previous in hope of improving the performance, and by the final version, a solid performance is reached
<br>
The link to the dataset used can be found in the project's README.

## Imports and Utils

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

from ml.layers.linear import Linear
from ml.layers.activations import Sigmoid, ReLU
from ml.loss import BCELoss, MSELoss
from ml.optimizers import SGD

### Loading data and creating data utils

In [2]:
# MNIST Dataset Class
class Dataset:
    def __init__(self, path):
        df = pd.read_csv(path, header=None)
        self.X = df.iloc[:, 1:]
        self.y = df.iloc[:, 0]

        self.X = self.X / 255

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return np.array(self.X.iloc[idx]), self.y[idx]

In [3]:
train_data = Dataset("./datasets/mnist/mnist_train.csv")
test_data = Dataset("./datasets/mnist/mnist_test.csv")
len(train_data), len(test_data)

(60000, 10000)

In [4]:
# Dataloader
def get_batch_loader(dataset, batchsize):
    n = len(dataset)
    indices = list(range(n))
    np.random.shuffle(indices)
    start = 0
    end = batchsize
    while True:
        if start == n:
            start = 0
            end = batchsize
            np.random.shuffle(indices)
            yield None
            continue
        batch_X, batch_y = list(), list()
        for i in range(start, end):
            X, y = train_data[i]
            batch_X.append(X)
            batch_y.append(y)
        yield np.array(batch_X), np.array(batch_y)
        start = end
        end = min(n, end + batchsize)

### Training and testing utilities

In [5]:
# Training utility
def train_model(model, loss_fn, dataloader, optimizer, n_epochs, grad_debug=False):
    for epoch in range(n_epochs):
        batch_count = 0
        grad_hist = list()
        while True:
            batch = next(dataloader)
            if not batch:
                break
            batch_count += 1
            x, y = batch
            y = np.eye(10)[y]
            y_pred = model.forward(x)
            loss = loss_fn.forward(y, y_pred)
            din_loss = loss_fn.backward()
            model.backward(din_loss)
            model.update(optimizer)
            if grad_debug:
                step_gradient = list()
                for l in model.layers:
                    if l.type == "learnable":
                        step_gradient.append(l.grad)
                grad_hist.append(step_gradient)
            print(
                "\rEpoch {}/{} Batch {} Loss {}".format(epoch+1, n_epochs, batch_count, loss), end=""
            )
            if np.isnan(loss):
                raise ValueError("Loss value has reached NaN, something is wrong")
            if np.isinf(loss):
                raise ValueError("Loss value has reached inf, something is wrong")
        print()
        if grad_debug:
            return grad_hist

In [6]:
# Testing utility
def test_model(model, dataloader):
    true = list()
    pred = list()
    while True:
        batch = next(dataloader)
        if not batch:
            break
        x, y = batch
        y_pred = np.argmax(clf.forward(x), axis=-1)
        true.extend(list(np.squeeze(y)))
        pred.extend(list(np.squeeze(y_pred)))
    print("Accuracy: {:.03f}%".format(100*accuracy_score(true, pred)))

### Model Class

In [7]:
# Base model class
class Model:
    def __init__(self, layers=[]):
        self.layers = layers

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)
        return x

    def backward(self, dx):
        for l in self.layers[::-1]:
            dx = l.backward(dx)

    def update(self, optim):
        for l in self.layers:
            if l.type == "learnable":
                for p in l.params:
                    l.params[p] = optim.update(l.params[p], l.grad[p])
                    
    def num_params(self):
        count = 0
        for l in self.layers:
            if l.type == "learnable":
                for name, params in l.params.items():
                    count += np.prod(params.shape)
        return count

## Implementation 1
- 3 dense layers followed by sigmoid
- MSE Loss
- SGD Optimizer with LR 0.1
- 5 epochs

A small network trained for a short duration to see how the learning goes. 

In [8]:
layers = [
    Linear(784, 100),
    Sigmoid(),
    Linear(100, 20),
    Sigmoid(),
    Linear(20, 10),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 80730


In [9]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [10]:
loss_fn = MSELoss()

In [11]:
sgd_optim = SGD(lr=0.1)

In [12]:
train_model(clf, loss_fn, train_loader, sgd_optim, 5)

Epoch 1/5 Batch 1875 Loss 0.047137227423678255
Epoch 2/5 Batch 1875 Loss 0.046330285603477725
Epoch 3/5 Batch 1875 Loss 0.046136397428807714
Epoch 4/5 Batch 1875 Loss 0.046091304782153106
Epoch 5/5 Batch 1875 Loss 0.046030348715061045


In [13]:
test_model(clf, train_loader)

Accuracy: 21.343%


In [14]:
test_model(clf, test_loader)

Accuracy: 21.840%


This small model performs very poorly, but then it was trained for a very short duration. The next implementation trains the same network for longer

## Implementation 2

- 3 dense layers followed by sigmoid
- MSE Loss
- SGD Optimizer with LR 0.1
- 10 epochs

This version trains the model for 10 epochs vs 5 epochs in previous version.

In [15]:
layers = [
    Linear(784, 100),
    Sigmoid(),
    Linear(100, 20),
    Sigmoid(),
    Linear(20, 10),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 80730


In [16]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [17]:
loss_fn = MSELoss()

In [18]:
sgd_optim = SGD(lr=0.1)

In [19]:
train_model(clf, loss_fn, train_loader, sgd_optim, 10)

Epoch 1/10 Batch 1875 Loss 0.07366473338410287
Epoch 2/10 Batch 1875 Loss 0.045338576624558556
Epoch 3/10 Batch 1875 Loss 0.044649676585136894
Epoch 4/10 Batch 1875 Loss 0.043895667895039875
Epoch 5/10 Batch 1875 Loss 0.043126566307594885
Epoch 6/10 Batch 1875 Loss 0.042386199181076694
Epoch 7/10 Batch 1875 Loss 0.041687917798274035
Epoch 8/10 Batch 1875 Loss 0.041029392827415854
Epoch 9/10 Batch 1875 Loss 0.040411216528821614
Epoch 10/10 Batch 1875 Loss 0.039836226156774685


In [20]:
test_model(clf, train_loader)

Accuracy: 31.860%


In [21]:
test_model(clf, test_loader)

Accuracy: 32.650%


This shows minor improvement but the performance is still poor. Something needs to change.
<br>
The first thing to change here is the loss function. MSE loss is not a suitable loss function for the task of classification. When paired with sigmoid activation, MSE loss can cause a slowdown in learning. In general, a cross-entropy loss is used. The next implementation replaces MSE with a BCE loss (binary cross-entropy)

## Implementation 3
- 3 dense layers followed by sigmoid
- BCE Loss
- SGD Optimizer with LR 0.1
- 5 epochs

In [22]:
layers = [
    Linear(784, 100),
    Sigmoid(),
    Linear(100, 20),
    Sigmoid(),
    Linear(20, 10),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 80730


In [23]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [24]:
loss_fn = BCELoss()

In [25]:
sgd_optim = SGD(lr=0.1)

In [26]:
train_model(clf, loss_fn, train_loader, sgd_optim, 10)

Epoch 1/10 Batch 1875 Loss 0.31522753038840196
Epoch 2/10 Batch 1875 Loss 0.29131345847025495
Epoch 3/10 Batch 1875 Loss 0.27427505168658585
Epoch 4/10 Batch 1875 Loss 0.25775393224504945
Epoch 5/10 Batch 1875 Loss 0.24097654013478956
Epoch 6/10 Batch 1875 Loss 0.22670178875852232
Epoch 7/10 Batch 1875 Loss 0.21343617909173368
Epoch 8/10 Batch 1875 Loss 0.20111324669733924
Epoch 9/10 Batch 1875 Loss 0.19074996649335482
Epoch 10/10 Batch 1875 Loss 0.18200842995863412


In [27]:
test_model(clf, train_loader)

Accuracy: 68.822%


In [28]:
test_model(clf, test_loader)

Accuracy: 69.260%


This is a very solid improvement. But still, there's some gap to cover. As the next step, the model is made larger and more powerful. 

## Implementation 4
- 10 dense layers followed by sigmoid
- BCE Loss
- SGD Optimizer with LR 0.1
- 10 epochs

In [29]:
layers = [
    Linear(784, 500),
    Sigmoid(),
    Linear(500, 300),
    Sigmoid(),
    Linear(300, 200),
    Sigmoid(),
    Linear(200, 100),
    Sigmoid(),
    Linear(100, 80),
    Sigmoid(),
    Linear(80, 60),
    Sigmoid(),
    Linear(60, 50),
    Sigmoid(),
    Linear(50, 20),
    Sigmoid(),
    Linear(20, 20),
    Sigmoid(),
    Linear(20, 10),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 640740


This number of parameters increases 8x from 80k to 640k.

In [30]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [31]:
loss_fn = BCELoss()

In [32]:
sgd_optim = SGD(lr=0.1)

In [33]:
train_model(clf, loss_fn, train_loader, sgd_optim, 10)

Epoch 1/10 Batch 1875 Loss 0.32065267100320665
Epoch 2/10 Batch 1875 Loss 0.30359540641930194
Epoch 3/10 Batch 1875 Loss 0.27758699601144066
Epoch 4/10 Batch 1875 Loss 0.24886533162364738
Epoch 5/10 Batch 1875 Loss 0.22131867472329345
Epoch 6/10 Batch 1875 Loss 0.19981650587817744
Epoch 7/10 Batch 1875 Loss 0.18165645233607336
Epoch 8/10 Batch 1875 Loss 0.16385136628219285
Epoch 9/10 Batch 1875 Loss 0.14793189293076198
Epoch 10/10 Batch 1875 Loss 0.13647412177745427


In [34]:
test_model(clf, train_loader)

Accuracy: 68.782%


In [35]:
test_model(clf, test_loader)

Accuracy: 68.870%


The scores did not improve. This is counterintuitive since larger networks should learn better. To understand the issue, the behavior of the gradients can be checked.

In [36]:
clf = Model(layers)

In [37]:
grad_history = train_model(clf, loss_fn, train_loader, sgd_optim, 1, grad_debug=True)

Epoch 1/1 Batch 1875 Loss 0.12595507255819066


In [38]:
w_norms = list()
b_norms = list()
for step_grad in grad_history:
    for layer_grad in step_grad:
        w_norms.append(np.linalg.norm(layer_grad['w']))
        b_norms.append(np.linalg.norm(layer_grad['b']))

In [39]:
print("Mean of grad-norm:", np.mean(w_norms), np.mean(b_norms))
print("Variance of grad-norm:", np.var(w_norms), np.var(b_norms))

Mean of grad-norm: 0.061534779654178674 0.008031953525073295
Variance of grad-norm: 0.0014953861889968982 1.254105387885096e-05


For the analysis. the gradients for all the parameters are stored for all the steps in the first epoch. The norm is taken for the gradients which reflects some measure of size. The mean and variance of the norms for w and b parameters are shown.
<br>
The numbers indicate that the gradients are small hinting at the problem of vanishing gradients. This arises in deeper networks with sigmoid activations, which tend to saturate making it slow and difficult to learn. The suggested activation function to tackle vanishing gradients is ReLU. The next implementation replaces all the sigmoid activations (except final) with ReLU.

## Implementation 5
- 10 dense layers, the first 9 followed by ReLU, last by Sigmoid
- BCE Loss
- SGD Optimizer with LR 0.1
- 10 epochs

In [40]:
layers = [
    Linear(784, 500),
    ReLU(),
    Linear(500, 300),
    ReLU(),
    Linear(300, 200),
    ReLU(),
    Linear(200, 100),
    ReLU(),
    Linear(100, 80),
    ReLU(),
    Linear(80, 60),
    ReLU(),
    Linear(60, 50),
    ReLU(),
    Linear(50, 20),
    ReLU(),
    Linear(20, 20),
    ReLU(),
    Linear(20, 10),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 640740


In [41]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [42]:
loss_fn = BCELoss()

In [43]:
sgd_optim = SGD(lr=0.1)

In [44]:
grad_history = train_model(clf, loss_fn, train_loader, sgd_optim, 1, grad_debug=True)

Epoch 1/1 Batch 1 Loss inf

/home/himansu/Programming/general_ml/ml-from-scratch/ml/layers/activations.py:24: RuntimeWarning: overflow encountered in exp
  return np.where(x >= 0, 1 / (1 + np.nan_to_num(np.exp(-x))), np.nan_to_num(np.exp(x)) / (1 + np.nan_to_num(np.exp(x))))
/home/himansu/Programming/general_ml/ml-from-scratch/ml/loss.py:40: RuntimeWarning: divide by zero encountered in log
  sample_loss = -(true * np.nan_to_num(np.log(pred)) + (1 - true) * np.nan_to_num(np.log(1 - pred)))
/home/himansu/.virtualenvs/general-ml-env/lib/python3.10/site-packages/numpy/core/_methods.py:179: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/himansu/Programming/general_ml/ml-from-scratch/ml/loss.py:46: RuntimeWarning: divide by zero encountered in true_divide
  return -(true / pred - (1 - true) / (1 - pred)) / (b * n)
/home/himansu/Programming/general_ml/ml-from-scratch/ml/loss.py:46: RuntimeWarning: invalid value encountered in true_divide
  return -(tru

ValueError: Loss value has reached inf, something is wrong

The loss value reaches infinity in the first iteration itself. To check what’s wrong, the model is initialized and a single step of forward-pass is taken.

In [45]:
clf_test = Model(layers)

In [46]:
x, y = next(train_loader)
y = np.eye(10)[y]
y_pred = clf.forward(x)

In [47]:
loss_fn.forward(y, y_pred)

inf

In [48]:
y_pred

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

All the predictions are zero. This happens with sigmoid activations when the input is very negative. This behavior can be caused by improper weight initialization. By default, the weights in the Linear layer are initialized from a unit normal distribution. The below implementation decreases the variance of the weights to 0.1.
<br>
But before that, the question is why did this happen with ReLU and didn’t happen with sigmoid? The outputs from the sigmoid are bounded to a range [0, 1]. So even if weights are large, the overall weighted sum is still small. When switching to ReLU, the unbounded nature of input changes to unbounded, which can result in very large activations.

In [49]:
def small_weight_init(val_range):
    def init(dims):
        in_d, out_d = dims
        return np.random.randn(in_d, out_d) * val_range
    return init

layers = [
    Linear(784, 500, small_weight_init(0.1)),
    ReLU(),
    Linear(500, 300, small_weight_init(0.1)),
    ReLU(),
    Linear(300, 200, small_weight_init(0.1)),
    ReLU(),
    Linear(200, 100, small_weight_init(0.1)),
    ReLU(),
    Linear(100, 80, small_weight_init(0.1)),
    ReLU(),
    Linear(80, 60, small_weight_init(0.1)),
    ReLU(),
    Linear(60, 50, small_weight_init(0.1)),
    ReLU(),
    Linear(50, 20, small_weight_init(0.1)),
    ReLU(),
    Linear(20, 20, small_weight_init(0.1)),
    ReLU(),
    Linear(20, 10, small_weight_init(0.1)),
    Sigmoid(),
]
clf = Model(layers)
print("Number of params:", clf.num_params())

Number of params: 640740


In [50]:
BATCHSIZE = 32
train_loader = get_batch_loader(train_data, BATCHSIZE)
test_loader = get_batch_loader(test_data, BATCHSIZE)

In [51]:
grad_history = train_model(clf, loss_fn, train_loader, sgd_optim, 1, grad_debug=True)

Epoch 1/1 Batch 1875 Loss 0.14808825166565814


With smaller weights, no instability was faced and the model trained without any issues. Now, we can check the behaviour of the gradients.

In [52]:
w_norms = list()
b_norms = list()
for step_grad in grad_history:
    for layer_grad in step_grad:
        w_norms.append(np.linalg.norm(layer_grad['w']))
        b_norms.append(np.linalg.norm(layer_grad['b']))

In [53]:
print("Mean of grad-norm:", np.mean(w_norms), np.mean(b_norms))
print("Variance of grad-norm:", np.var(w_norms), np.var(b_norms))

Mean of grad-norm: 0.15127703638150822 0.008138200530934742
Variance of grad-norm: 0.002460627057115184 3.668307440557543e-05


The means of gradient norms are now much larger. This suggests that the learning slowdown shouldn't be happening here. This version is trained for 9 further epochs (since 1 epoch is already trained).

In [54]:
train_model(clf, loss_fn, train_loader, sgd_optim, 9)

Epoch 1/9 Batch 1875 Loss 0.0164925605651016426
Epoch 2/9 Batch 1875 Loss 0.0069168688169994466
Epoch 3/9 Batch 1875 Loss 0.0020783844567613804
Epoch 4/9 Batch 1875 Loss 0.00151930843760857617
Epoch 5/9 Batch 1875 Loss 0.00108543128782836725
Epoch 6/9 Batch 1875 Loss 0.00129777724346944255
Epoch 7/9 Batch 1875 Loss 0.00334526778004999451
Epoch 8/9 Batch 1875 Loss 0.00120160082590837105
Epoch 9/9 Batch 1875 Loss 0.00076900947263972114


In [55]:
test_model(clf, train_loader)

Accuracy: 99.002%


In [56]:
test_model(clf, test_loader)

Accuracy: 99.000%


This version crosses 99% accuracy- a very huge improvement. The first implementation started from accuracy in the 20s, and after incremental improvements, has reached almost perfect accuracy.